In [1]:
import pickle
import pandas as pd
import os

In [8]:
dataset_dir = "/vol3/sunil/FinalDataset/videos/"
file_list = [os.path.splitext(f)[0] for f in os.listdir(dataset_dir)]

In [12]:
# final_df_list = []
# for f in file_list:
#     encoding_df_dir = f"/home/sunil/projects/Stuff/Combined/WhoSaidThat/output/video_temp/{f}/pywork/encoding_df.pckl"
#     encoding_df = pickle.load(open(encoding_df_dir, "rb"))
#     encoding_df["file_name"] = f
#     encoding_df = encoding_df.drop('Encoding', axis=1)
#     final_df_list.append(encoding_df)
    
# final_df = pd.concat(final_df_list)
# final_df.to_excel("../../../del_later/FaceClusteringAnnotations.xlsx", index=False)

In [45]:
def match_output(output_df, gt_col, pred_col, matched_col):
    
    output_df = output_df.copy()

    pred_list = output_df[pred_col].unique()
    pred_match = {}
    pred_count = {}

    for pred in pred_list:
        if pred == "Unassigned":
            pred_match[pred] = "Unassigned"
        else:
            pred_match[pred] = output_df[output_df[pred_col] == pred][gt_col].value_counts().idxmax()
            pred_count[pred] = len(output_df[output_df[pred_col] == pred])

    temp_count = {}

    for pred, gt in pred_match.items():
        if gt in temp_count.keys():
            temp_count[gt].append(pred)
        else:
            temp_count[gt] = [pred]

    for gt, preds in temp_count.items():
        if len(preds) > 1:
            max_count = 0
            max_match = None
            for pred in preds:
                if pred_count[pred] > max_count:
                    if max_match is None:
                        max_match = pred
                        max_count = pred_count[pred]
                    else:
                        pred_match[max_match] = "Unassigned"
                        max_match = pred
                        max_count = pred_count[pred]
                else:
                    pred_match[pred] = "Unassigned"
            pred_match[max_match] = gt

    output_df[matched_col] = output_df.apply(
        lambda row: pred_match[row[pred_col]],
        axis=1,
    )

    return output_df


def compute_performance(output_df, gt_col, pred_col, matched_col):

    def _match(gt, pred, matched):
        if pred == "Unassigned":
            return "Missed"
        elif matched == "Unassigned" or matched != gt:
            return "Confusion"
        else:
            return "Correct"

    output_df["matched_result"] = output_df.apply(
        lambda row: _match(row[gt_col], row[pred_col], row[matched_col]),
        axis=1,
    )

    return output_df

In [17]:
fa_df = pd.read_excel("../../../del_later/FaceClusteringAnnotations.xlsx")
fa_df

,Filename,Track,GT,Marked
0,StarTalk_Cosmic_780_850,0,Speaker_01,1
1,StarTalk_Cosmic_780_850,1,Speaker_02,2
2,StarTalk_Cosmic_780_850,2,Speaker_01,1
3,StarTalk_Cosmic_780_850,3,Speaker_03,3
4,StarTalk_Cosmic_780_850,4,Speaker_01,1
...,...,...,...,...
962,StarTalk_Farming_1700_1800,8,Speaker_01,1
963,StarTalk_Farming_1700_1800,9,Speaker_03,3
964,StarTalk_Farming_1700_1800,10,Speaker_02,2
965,StarTalk_Farming_1700_1800,11,Speaker_04,4


In [85]:
track_cluster_df_list = []
for f in file_list:
    encoding_df_dir = f"/home/sunil/projects/Stuff/Combined/WhoSaidThat/output/video_temp/{f}/pywork/encoding_df_w_clustering.pckl"
    encoding_df = pickle.load(open(encoding_df_dir, "rb"))
    encoding_df["Filename"] = f
    encoding_df = encoding_df.drop('Encoding', axis=1)
    track_cluster_df_list.append(encoding_df)
    
track_cluster_df = pd.concat(track_cluster_df_list)

In [86]:
track_cluster_df

,Frame,Track,Score,S,X,Y,SimpleClusters_wo_Track,SimpleClusters,Filename
0,0,0,1.733333,222.861984,906.291321,474.285416,0,0,StarTalk_Cosmic_780_850
1,1,0,1.825000,224.087524,906.291321,474.748413,0,0,StarTalk_Cosmic_780_850
2,2,0,2.040000,224.105255,906.291321,476.064102,0,0,StarTalk_Cosmic_780_850
3,3,0,2.300000,224.292931,906.291321,476.338852,0,0,StarTalk_Cosmic_780_850
4,4,0,2.560000,224.339714,906.291321,476.338852,0,0,StarTalk_Cosmic_780_850
...,...,...,...,...,...,...,...,...,...
1498,2496,12,2.420000,315.949226,992.813812,549.539879,1,1,StarTalk_Farming_1700_1800
1499,2497,12,2.225000,315.949226,992.568481,548.523827,1,1,StarTalk_Farming_1700_1800
1500,2498,12,2.000000,315.949226,992.262482,545.957558,1,1,StarTalk_Farming_1700_1800
1501,2499,12,1.700000,315.941658,992.025146,545.810181,1,1,StarTalk_Farming_1700_1800


In [87]:
track_cluster_df = track_cluster_df[["Filename", "Frame", "Track", "SimpleClusters", "SimpleClusters_wo_Track"]]

In [88]:
track_cluster_df

,Filename,Frame,Track,SimpleClusters,SimpleClusters_wo_Track
0,StarTalk_Cosmic_780_850,0,0,0,0
1,StarTalk_Cosmic_780_850,1,0,0,0
2,StarTalk_Cosmic_780_850,2,0,0,0
3,StarTalk_Cosmic_780_850,3,0,0,0
4,StarTalk_Cosmic_780_850,4,0,0,0
...,...,...,...,...,...
1498,StarTalk_Farming_1700_1800,2496,12,1,1
1499,StarTalk_Farming_1700_1800,2497,12,1,1
1500,StarTalk_Farming_1700_1800,2498,12,1,1
1501,StarTalk_Farming_1700_1800,2499,12,1,1


In [89]:
temp_df = track_cluster_df.merge(fa_df)

In [94]:
temp_df

,Filename,Frame,Track,SimpleClusters,SimpleClusters_wo_Track,GT,Marked
0,StarTalk_Cosmic_780_850,0,0,0,0,Speaker_01,1
1,StarTalk_Cosmic_780_850,1,0,0,0,Speaker_01,1
2,StarTalk_Cosmic_780_850,2,0,0,0,Speaker_01,1
3,StarTalk_Cosmic_780_850,3,0,0,0,Speaker_01,1
4,StarTalk_Cosmic_780_850,4,0,0,0,Speaker_01,1
...,...,...,...,...,...,...,...
132939,StarTalk_Farming_1700_1800,2496,12,1,1,Speaker_01,1
132940,StarTalk_Farming_1700_1800,2497,12,1,1,Speaker_01,1
132941,StarTalk_Farming_1700_1800,2498,12,1,1,Speaker_01,1
132942,StarTalk_Farming_1700_1800,2499,12,1,1,Speaker_01,1


In [95]:
temp_list = []
for f in file_list:
    matched_df = match_output(temp_df[temp_df["Filename"] == f], gt_col="GT", pred_col="SimpleClusters", matched_col="Matched_Output")
    matched_df = compute_performance(
            output_df=matched_df, gt_col="GT", pred_col="SimpleClusters", matched_col="Matched_Output"
        )
    temp = matched_df[["GT", "matched_result", "SimpleClusters"]].groupby(
        ["GT", "matched_result"]).count().reset_index().pivot_table(
        'SimpleClusters', "GT", 'matched_result').fillna(0).astype(int).reset_index().rename_axis(None, axis=1)
    temp["Filename"] = f
    temp = temp.fillna(0)
    temp_list.append(temp)
    
face_clustering_results = pd.concat(temp_list)
face_clustering_results = face_clustering_results.fillna(0)
face_clustering_results["Confusion"] = face_clustering_results["Confusion"].astype(int)

In [96]:
temp_list = []
for f in file_list:
    matched_df = match_output(temp_df[temp_df["Filename"] == f], gt_col="GT", pred_col="SimpleClusters_wo_Track", matched_col="Matched_Output")
    matched_df = compute_performance(
            output_df=matched_df, gt_col="GT", pred_col="SimpleClusters_wo_Track", matched_col="Matched_Output"
        )
    temp = matched_df[["GT", "matched_result", "SimpleClusters_wo_Track"]].groupby(
        ["GT", "matched_result"]).count().reset_index().pivot_table(
        'SimpleClusters_wo_Track', "GT", 'matched_result').fillna(0).astype(int).reset_index().rename_axis(None, axis=1)
    temp["Filename"] = f
    temp = temp.fillna(0)
    temp_list.append(temp)
    
face_clustering_results_wo_track = pd.concat(temp_list)
face_clustering_results_wo_track = face_clustering_results_wo_track.fillna(0)
face_clustering_results_wo_track["Confusion"] = face_clustering_results_wo_track["Confusion"].astype(int)

In [97]:
face_clustering_results

,GT,Correct,Filename,Confusion
0,Speaker_01,607,StarTalk_Cosmic_780_850,0
1,Speaker_02,451,StarTalk_Cosmic_780_850,0
2,Speaker_03,298,StarTalk_Cosmic_780_850,0
0,Speaker_01,960,StarTalk_Mars_1680_1780,0
1,Speaker_02,700,StarTalk_Mars_1680_1780,0
...,...,...,...,...
3,Speaker_04,629,StarTalk_Farming_2550_2645,0
0,Speaker_01,609,StarTalk_Farming_1700_1800,0
1,Speaker_02,77,StarTalk_Farming_1700_1800,0
2,Speaker_03,208,StarTalk_Farming_1700_1800,0


In [101]:
final_results = face_clustering_results.merge(face_clustering_results_wo_track, on=["GT", "Filename"], suffixes=('', '_wo_track'))

In [102]:
final_results.to_excel("../../../del_later/face_clustering_results.xlsx", index=False)

In [103]:
# Overlap analysis

In [116]:
def has_overlap(col_list, row):
    
    col_count = 0
    
    for col_name in col_list:
        if row[col_name] > 0:
            col_count += 1
            
    if col_count > 1:
        return True
    
    return False

In [122]:
temp_list = []
for f in file_list:
    file_loc = f"/home/sunil/projects/Stuff/Combined/WhoSaidThat/output/run_output/speaker_stats/{f}_additional_info.xlsx"
    
    temp_df = pd.read_excel(file_loc)
    
    col_list = [col for col in temp_df.columns if "audio" in col]
    
    temp_df["has_overlap"] = temp_df.apply(lambda row: has_overlap(col_list, row), axis=1)
    
    temp_df = temp_df[["word", "word_start", "word_end", "speaker", "has_overlap"]]
    
    temp_df["file_name"] = f
    
    temp_list.append(temp_df)
    
word_overlap_status = pd.concat(temp_list)

In [123]:
word_overlap_status

,word,word_start,word_end,speaker,has_overlap,file_name
0,but,0.00,0.16,Delilah Gates,False,StarTalk_Cosmic_780_850
1,typically,0.16,0.82,Delilah Gates,False,StarTalk_Cosmic_780_850
2,stability,0.82,1.18,Delilah Gates,False,StarTalk_Cosmic_780_850
3,is,1.18,1.52,Delilah Gates,False,StarTalk_Cosmic_780_850
4,not,1.52,1.68,Delilah Gates,False,StarTalk_Cosmic_780_850
...,...,...,...,...,...,...
241,or,98.58,98.70,Neil deGrasse Tyson,False,StarTalk_Farming_1700_1800
242,not,98.82,99.00,Neil deGrasse Tyson,False,StarTalk_Farming_1700_1800
243,fast,99.00,99.14,Neil deGrasse Tyson,False,StarTalk_Farming_1700_1800
244,at,99.30,99.56,Neil deGrasse Tyson,False,StarTalk_Farming_1700_1800


In [127]:
temp_list = []
for f in file_list:    
    file_loc = f"/home/sunil/projects/Stuff/Combined/WhoSaidThat/output/run_output/speaker_stats/{f}_audio_12.xlsx"
    temp_df = pd.read_excel(file_loc)
    
    temp_df = temp_df[["file_name", "word", "word_start", "word_end", "matched_result"]].rename(columns={"matched_result": "audio_result"})
    temp_df["file_name"] = temp_df["file_name"].apply(lambda x: os.path.splitext(x)[0])
    
    temp_list.append(temp_df)
    
audio_result = pd.concat(temp_list)

In [128]:
audio_result

,file_name,word,word_start,word_end,audio_result
0,StarTalk_Cosmic_780_850,but,0.00,0.16,Correct
1,StarTalk_Cosmic_780_850,typically,0.16,0.82,Correct
2,StarTalk_Cosmic_780_850,stability,0.82,1.18,Correct
3,StarTalk_Cosmic_780_850,is,1.18,1.52,Correct
4,StarTalk_Cosmic_780_850,not,1.52,1.68,Correct
...,...,...,...,...,...
241,StarTalk_Farming_1700_1800,or,98.58,98.70,Correct
242,StarTalk_Farming_1700_1800,not,98.82,99.00,Correct
243,StarTalk_Farming_1700_1800,fast,99.00,99.14,Correct
244,StarTalk_Farming_1700_1800,at,99.30,99.56,Correct


In [129]:
temp_list = []
for f in file_list:    
    file_loc = f"/home/sunil/projects/Stuff/Combined/WhoSaidThat/output/run_output/speaker_stats/{f}_video_simple.xlsx"
    temp_df = pd.read_excel(file_loc)
    
    temp_df = temp_df[["file_name", "word", "word_start", "word_end", "matched_result"]].rename(columns={"matched_result": "video_result"})
    temp_df["file_name"] = temp_df["file_name"].apply(lambda x: os.path.splitext(x)[0])
    
    temp_list.append(temp_df)
    
video_result = pd.concat(temp_list)

In [130]:
video_result

,file_name,word,word_start,word_end,video_result
0,StarTalk_Cosmic_780_850,but,0.00,0.16,Correct
1,StarTalk_Cosmic_780_850,typically,0.16,0.82,Correct
2,StarTalk_Cosmic_780_850,stability,0.82,1.18,Correct
3,StarTalk_Cosmic_780_850,is,1.18,1.52,Correct
4,StarTalk_Cosmic_780_850,not,1.52,1.68,Correct
...,...,...,...,...,...
241,StarTalk_Farming_1700_1800,or,98.58,98.70,Correct
242,StarTalk_Farming_1700_1800,not,98.82,99.00,Correct
243,StarTalk_Farming_1700_1800,fast,99.00,99.14,Correct
244,StarTalk_Farming_1700_1800,at,99.30,99.56,Correct


In [131]:
audio_result.to_excel("Final_Speaker_Assignments_Audio.xlsx", index=False)
video_result.to_excel("Final_Speaker_Assignments_Video.xlsx", index=False)

In [132]:
word_overlap_status.to_excel("../../../del_later/Word_Overlap_Status.xlsx", index=False)

In [3]:
# Visualizations

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
import plotly.graph_objects as go
stats_df = pd.read_excel("../../Dataset/Dataset_stats.xlsx")

In [2]:
stats_df

,Video Name,Video Type,Video Length,Number of words,Off-screen words,Audio Error,Video Error,Combined Error,Video Missed Error,Video Incorrect,Change,Overlap %,Face Clusterng error,Word overlap %
0,ESPN_1,In-person,212.161,595,322,0.289076,0.645378,0.290756,0.541176,0.104202,-0.001681,0.097038,0.1334,0.183193
1,MagnusCarlson_542_599,In-person,57.051,158,8,0.145570,0.050633,0.012658,0.050633,0.000000,0.132911,0.007744,0.0168,0.025316
2,NDT_India_19_88,In-person,69.056,220,72,0.031818,0.345455,0.063636,0.327273,0.018182,-0.031818,0.029656,0.0008,0.040909
3,PiersMorgan_1_0_165,In-person,165.001,608,158,0.157895,0.396382,0.208882,0.259868,0.136513,-0.050987,0.161957,0.1116,0.312500
4,PiersMorgan_1_165_368,In-person,203.012,750,225,0.233333,0.456000,0.233333,0.300000,0.156000,0.000000,0.282972,0.0049,0.517333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,StarTalk_Sleep_2470_2551,Virtual,81.014,248,39,0.137097,0.165323,0.250000,0.157258,0.008065,-0.112903,0.026709,0.0000,0.036290
72,StarTalk_Sleep_382_450,Virtual,68.104,166,16,0.186747,0.114458,0.024096,0.096386,0.018072,0.162651,0.095431,0.0000,0.120482
73,StarTalk_Sleep_748_796,Virtual,48.019,122,29,0.172131,0.254098,0.106557,0.237705,0.016393,0.065574,0.137620,0.0000,0.237705
74,TalkSport_1_0_131,In-person,131.007,412,114,0.019417,0.279126,0.002427,0.276699,0.002427,0.016990,0.004645,0.0033,0.014563


In [3]:
stats_df_2 = pd.read_excel("../../Dataset/Dataset_stats.xlsx", sheet_name="Sheet4")
temp = stats_df_2[["Video Name", "Number of Speakers"]]
temp

,Video Name,Number of Speakers
0,ESPN_1,5
1,MagnusCarlson_542_599,2
2,NDT_India_19_88,4
3,PiersMorgan_1_0_165,4
4,PiersMorgan_1_165_368,4
...,...,...
71,StarTalk_Sleep_2470_2551,3
72,StarTalk_Sleep_382_450,4
73,StarTalk_Sleep_748_796,4
74,TalkSport_1_0_131,3


In [8]:
temp_df = stats_df[stats_df["Video Type"] == "Virtual"].merge(temp)

data = {
    'Date': temp_df.index,
    'Commodity1': temp_df["Face Clusterng error"],
    'Commodity2': temp_df["Word overlap %"],
    'Commodity3': temp_df["Video Incorrect"],
    'Commodity4': temp_df["Number of Speakers"]
}
df = pd.DataFrame(data)

df.sort_values(["Commodity2"], inplace=True)
df.reset_index(drop=True, inplace=True)

df["Date"] = df.index + 1

# Normalization
# df['Commodity1_norm'] = df['Commodity1'] / df['Commodity1'].max()
# df['Commodity2_norm'] = df['Commodity2'] / df['Commodity2'].max()
# df['Commodity3_norm'] = df['Commodity3'] / df['Commodity3'].max()
df['Commodity1_norm'] = gaussian_filter1d(df['Commodity1'], sigma=2)
df['Commodity2_norm'] = gaussian_filter1d(df['Commodity2'], sigma=2)
df['Commodity3_norm'] = gaussian_filter1d(df['Commodity3'], sigma=2)

# Plot
# plt.figure(figsize=(15, 6))
# plt.plot(df['Date'], df['Commodity1_norm'], label='Commodity 1')
# plt.plot(df['Date'], df['Commodity2_norm'], label='Commodity 2')
# plt.plot(df['Date'], df['Commodity3_norm'], label='Commodity 3')
# plt.xlabel('Date')
# plt.ylabel('Normalized Value')
# plt.title('Normalized Trends of Commodities')
# plt.legend()
# plt.show()

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=df['Date'], y=df['Commodity1_norm'], mode='lines', name='CER'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Commodity2_norm'], mode='lines', name='Word overlap %'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Commodity3_norm'], mode='lines', name='Video Incorrect MWDE'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Commodity4'], mode='lines', name='Num Speakers'))

# Update layout
fig.update_layout(
#     title='Trends comparison of CER, word overlap % and video incorrect MWDE',
    xaxis=dict(title='Video Number'),
    yaxis=dict(title='Smoothed Value'),
    legend=dict(x=0, y=1.1, orientation='h')
)

# Show the plot
fig.show()

In [13]:
import plotly.express as px
import pandas as pd

# Creating a scatter plot
fig = px.scatter(df, x='Commodity3', y='Commodity2', title='Scatter Plot of Quantity_X vs Quantity_Y')

# Showing the plot
fig.show()


In [6]:
fig.write_image("../../../del_later/vosd_trend.png")